In [ ]:
from wobl_sim.robot import Robot
from dm_control.locomotion.arenas import Floor
from dm_control import mjcf
import numpy as np
import control
import mujoco
import mediapy as media

# Function to compute the full-body Center of Mass
def compute_total_com(model, data):
    total_mass = 0.0
    com = np.zeros(3)

    for i in range(model.nbody):
        body_mass = model.body_mass[i]
        # Get world position of this body's CoM
        body_com = data.xipos[i]
        com += body_mass * body_com
        total_mass += body_mass

    return com / total_mass if total_mass > 0 else com

def find_equilibrium(robot: Robot, physics: mjcf.Physics):
    com = compute_total_com(physics.model, physics.data)
    com_x_offset = com[0]
    com_z_offset = com[2] - robot.mjcf_model.find("joint", "L_foot").pos[2]
    theta_eq = -com_x_offset / com_z_offset

    # Apply small-angle to root freejoint quaternion
    qpos = physics.data.qpos.copy()
    qpos[4] = theta_eq / 2  # qx component of quaternion
    physics.data.qpos[:] = qpos
    return theta_eq

robot = Robot("../mjcf")
arena = Floor(reflectance=0.0)
arena.add_free_entity(robot)

physics = mjcf.Physics.from_mjcf_model(arena.mjcf_model)
physics.model.opt.timestep = 0.005
dt = physics.model.opt.timestep
print("time stepp:", dt)
#robot.set_pose(physics, np.array([0, 0, 0.237]), np.array([1, 0, 0, 0]))


In [ ]:
def print_model_info(mjcf_model: mjcf.RootElement, model):
    print("=== Model Info ===")
    print("nq =", model.nq, " nv =", model.nv, " nu =", model.nu)
    print()

    # Joint names and their qpos/qvel addresses
    print("=== Joints ===")
    for i, jn in enumerate(mjcf_model.find_all("joint")):
        addr_qpos = model.jnt_qposadr[i] if i < len(model.jnt_qposadr) else None
        addr_qvel = model.jnt_dofadr[i] if i < len(model.jnt_dofadr) else None
        print(f"Joint {i}: {jn}  qpos[{addr_qpos}]  qvel[{addr_qvel}]")

    print()

    # Actuators
    print("=== Actuators ===")
    for i, an in enumerate(mjcf_model.find_all("actuator")):
        trnid = model.actuator_trnid[i]
        joint_id = trnid[0]
        print(f"Actuator {i}: {an} -> joint {joint_id}")
print_model_info(arena.mjcf_model, physics.model)

In [ ]:
physics.reset()
theta_eq = find_equilibrium(robot, physics)
print(f"Approx equilibrium pitch (rad): {theta_eq:.4f}")
#robot.set_pose(physics, np.array([0, 0, 0.24]), np.array([1, 0, 0, 0]))
physics.forward()
physics.data.qacc[:] = 0  # Assert that there is no the acceleration.
mujoco.mj_inverse(physics.model.ptr, physics.data.ptr)
print(physics.data.qfrc_inverse)

In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

def set_robot_pose(robot, physics, offset, theta_eq):
    physics.reset()
    quat = Rotation.from_euler('y', theta_eq).as_quat(scalar_first=True)  # Quaternion representing pitch rotation
    robot.set_pose(physics, np.array([0, 0, offset]), quat)
    physics.forward()

height_offsets = np.linspace(0.23, 0.24, 10000)
vertical_forces = []
for offset in height_offsets:
    #physics.reset()
    #find_equilibrium(robot, physics)
    #physics.forward()
    set_robot_pose(robot, physics, offset, theta_eq)
    physics.data.qacc[:] = 0
    # Offset the height by `offset`.
    physics.data.qpos[2] = offset
    #find_equilibrium(robot, physics)
    mujoco.mj_inverse(physics.model.ptr, physics.data.ptr)
    vertical_forces.append(physics.data.qfrc_inverse[2])

# Find the height-offset at which the vertical force is smallest.
idx = np.argmin(np.abs(vertical_forces))
best_offset = height_offsets[idx]

# Plot the relationship.
plt.figure(figsize=(10, 6))
plt.plot(height_offsets * 1000, vertical_forces, linewidth=3)
# Red vertical line at offset corresponding to smallest vertical force.
plt.axvline(x=best_offset * 1000, color="red", linestyle="--")
weight = physics.model.body_subtreemass[1] * np.linalg.norm(physics.model.opt.gravity)
plt.axhline(y=weight, color="green", linestyle="--")
plt.xlabel("Height offset (mm)")
plt.ylabel("Vertical force (N)")
plt.grid(which="major", color="#DDDDDD", linewidth=0.8)
plt.grid(which="minor", color="#EEEEEE", linestyle=":", linewidth=0.5)
plt.minorticks_on()
plt.title(f"Smallest vertical force " f"found at offset {best_offset*1000:.4f}mm.")
plt.show()

In [ ]:
DURATION  = 3   # seconds
FRAMERATE = 60  # Hz
set_robot_pose(robot, physics, best_offset, theta_eq)

model, data = physics.model, physics.data
renderer = mujoco.Renderer(model.ptr, height=480, width=640)

frames = []
while data.time < DURATION:
    # Step the simulation.
    physics.step()

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data.ptr)
        pixels = renderer.render()
        frames.append(pixels)

# Display video.
media.show_video(frames, fps=FRAMERATE)
renderer.close()

In [ ]:
set_robot_pose(robot, physics, best_offset, theta_eq)
mujoco.mj_inverse(physics.model.ptr, physics.data.ptr)
print(physics.data.qfrc_inverse[2])
nq = physics.model.nq       # number of generalized coords
nv = physics.model.nv       # number of velocities
nu = physics.model.nu       # number of actuators
nx = nv * 2                 # state vector [qpos, qvel]
print("nq =", nq, " nv =", nv, " nu =", nu)

# states = [qpos; qvel], input = actuators
# Allocate matrices for discrete-time linearization
A = np.zeros((nx, nx))
B = np.zeros((nx, nu))


# mjd_transitionFD computes [A,B] around (qpos,qvel,u)
# It expects data to be at the linearization point
eps = 1e-4
flg_centered = True
mujoco.mjd_transitionFD(physics.model.ptr, physics.data.ptr, eps, flg_centered, A, B, None, None)

# --- Reduced state: [pitch, pitch_rate, forward_velocity] ---
# Freejoint indices: qpos[4] = qx ~ pitch (small-angle)
# qvel[3] = angular velocity x (pitch rate)
# qvel[0] = linear velocity x (forward)
idx_theta = 4
idx_theta_dot = nv + 4
idx_vx = nv

S = np.zeros((3, nx))
S[0, idx_theta] = 1.0
S[1, idx_theta_dot] = 1.0
S[2, idx_vx] = 1.0

A_reduced = S @ A @ S.T

# --- Inputs: wheel velocity actuators ---
wheel_actuators = [2, 3]  # L_foot, R_foot
B_reduced = S @ B
B_reduced = B_reduced[:, wheel_actuators]
B_scale = 1e2  # Scale factor to improve numerical conditioning
B_reduced_scaled = B_reduced * B_scale

print("A reduced")
print(A_reduced)
print("B reduced")
print(B_reduced )

dt = physics.model.opt.timestep
Ac = (A_reduced - np.eye(3)) / dt
Bc = B_reduced / dt

# --- LQR ---
Q = np.diag([10, 5, 1])  # states: pitch, pitch_rate, forward velocity
R = np.eye(2) * 10.0        # wheel velocity effort
#K, S_sol, E = control.dlqr(A_reduced, B_reduced_scaled, Q, R)
K, S_sol, E = control.lqr(Ac, Bc, Q, R)
K[:,0] = 0.5 * (K[0,0] + K[1,0])
K[:,1] = 0.5 * (K[0,1] + K[1,1])
K[:,2] = 0.5 * (K[0,2] + K[1,2])
K[1,:] = K[0,:]  # make L/R gains identical
print("LQR gain K:\n", K)

In [ ]:
set_robot_pose(robot, physics, best_offset, theta_eq)

model, data = physics.model, physics.data
qpos0 = data.qpos.copy()
ctrl0 = data.ctrl.copy()
# Parameters.
DURATION = 4  # seconds
FRAMERATE = 30  # Hz

# Allocate position difference dq.
dq = np.zeros(model.nv)

renderer = mujoco.Renderer(model.ptr, height=480, width=640)
frames = []
ctrls = []
states = []
while data.time < DURATION:
    # Generalized position difference.
    mujoco.mj_differentiatePos(model.ptr, dq, 1, qpos0, data.qpos)
    full_state = np.hstack((dq, data.qvel))  # shape (nx,)

    # Project into reduced coordinates: [theta, theta_dot, vx]
    #x_red = S @ full_state
    x_red = np.array([dq[4], data.qvel[4], data.qvel[0]])
    #x_red[2] *= -1

    # LQR control (only wheel actuators)
    u_red = -K @ x_red * dt

    # Apply to wheel actuators (indices [2,3])
    data.ctrl[:] = ctrl0  # reset control
    data.ctrl[2] = u_red[0] + 5
    data.ctrl[3] = u_red[1] + 5
    # Step the simulation.
    physics.step()

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data.ptr)
        pixels = renderer.render()
        frames.append(pixels)
        ctrls.append(data.ctrl.copy())
        states.append(full_state)

print(u_red)
print(x_red)
print(S @ full_state)
#print(states[-1])
media.show_video(frames, fps=FRAMERATE)
renderer.close()

In [ ]:
import numpy as np
# If you used discrete A_reduced, B_reduced:
Ad = A_reduced.copy()
Bd = B_reduced.copy()
# closed-loop discrete:
Acl = Ad - Bd @ K
eig = np.linalg.eigvals(Acl)
print("Discrete closed-loop eigenvalues:", eig)
print("Max magnitude:", np.max(np.abs(eig)))

# 1) eigenvalues
Acl = Ad - Bd @ K
eig = np.linalg.eigvals(Acl)
print("Closed-loop eigenvalues (discrete):", eig)
print("Max |eig|:", np.max(np.abs(eig)))

# 2) discrete linear sim
x0 = np.array([np.deg2rad(2.0), 0.0, 0.0])  # 2 deg initial lean
x = x0.copy()
for i in range(200):
    u = -K @ x                 # control
    x = Ad @ x + Bd @ u
print("Linear sim after 200 steps:", x)

In [ ]:
import copy
model, data = physics.model, physics.data

def empirical_B_col(actuator_index, du=1e-3):
    qpos_base = data.qpos.copy()
    qvel_base = data.qvel.copy()
    ctrl_base = data.ctrl.copy()

    # baseline forward one step to ensure consistent dynamics (optional)
    physics.step()

    # apply +du to actuator for one step and measure Δx (reduced)
    data.ctrl[actuator_index] += du
    physics.step()
    mujoco.mj_differentiatePos(model.ptr, dq, 1, qpos0, data.qpos)  # reuse your dq/qpos0
    full_state = np.hstack((dq, data.qvel))
    x_plus = S @ full_state

    # revert to base pose
    data.qpos[:] = qpos_base
    data.qvel[:] = qvel_base
    data.ctrl[:] = ctrl_base
    physics.forward()

    # apply -du
    data.ctrl[actuator_index] -= du
    physics.step()
    mujoco.mj_differentiatePos(model.ptr, dq, 1, qpos0, data.qpos)
    full_state = np.hstack((dq, data.qvel))
    x_minus = S @ full_state

    # restore
    data.qpos[:] = qpos_base
    data.qvel[:] = qvel_base
    data.ctrl[:] = ctrl_base
    physics.forward()

    Bd_col_est = (x_plus - x_minus) / (2*du)   # central diff estimate
    return Bd_col_est

dq = np.zeros(model.nv)
qpos0 = data.qpos.copy()

col0 = empirical_B_col(2, du=1e-3)  # L wheel
col1 = empirical_B_col(3, du=1e-3)  # R wheel
print("Empirical Bd columns:", col0, col1)
print("Analytic Bd columns:", Bd[:,0], Bd[:,1])